
Import Dependedncies



In [120]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score

Data colleaction and analysis

In [121]:
# load csv file to create dataframe
diabetes_data = pd.read_csv('/content/diabetes.csv')

In [122]:
#print first 5 rows of dataframe
diabetes_data.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0


In [123]:
# Give the shape of the dataframe
diabetes_data.shape

(100000, 9)

In [124]:
# Get standard mesurements
diabetes_data.describe()

,age,hypertension,heart_disease,bmi,HbA1c_level,blood_glucose_level,diabetes
count,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,41.885856,0.07485,0.039420,27.320767,5.527507,138.058060,0.085000
std,22.516840,0.26315,0.194593,6.636783,1.070672,40.708136,0.278883
min,0.080000,0.00000,0.000000,10.010000,3.500000,80.000000,0.000000
25%,24.000000,0.00000,0.000000,23.630000,4.800000,100.000000,0.000000
50%,43.000000,0.00000,0.000000,27.320000,5.800000,140.000000,0.000000
75%,60.000000,0.00000,0.000000,29.580000,6.200000,159.000000,0.000000
max,80.000000,1.00000,1.000000,95.690000,9.000000,300.000000,1.000000


In [125]:
#Value counts
diabetes_data['diabetes'].value_counts()

,count
diabetes,
0,91500
1,8500


From this dataset, 8500 people are diabetic and 91500 people are not

In [126]:
diabetes_data['gender'].value_counts()

,count
gender,
Female,58552
Male,41430
Other,18


There are 58552 women and 41430 men

In [127]:
group_by_gender = diabetes_data.groupby('gender')

females = group_by_gender.get_group('Female')
males = group_by_gender.get_group('Male')

# Diabetic scores on Females
print('Females: ',females['diabetes'].value_counts())

# Diabetic sscores on Males
print('Males: ', males['diabetes'].value_counts())

Females:  diabetes
0    54091
1     4461
Name: count, dtype: int64
Males:  diabetes
0    37391
1     4039
Name: count, dtype: int64


4461 out of 58552 women are diabetic

4093 out of 41430 men are diabetic

In [128]:
#Mean value of each column after gtouping by diabetes column
diabetes_data.drop(columns=['gender', 'smoking_history'], axis=1).groupby('diabetes').mean()

,age,hypertension,heart_disease,bmi,HbA1c_level,blood_glucose_level
diabetes,,,,,,
0,40.115187,0.058984,0.029235,26.887163,5.396761,132.852470
1,60.946588,0.245647,0.149059,31.988382,6.934953,194.094706


The average diabetic in this data is 60 years of age with a higher HbA1c level, glucose level and hypertesion level

In [129]:
A = diabetes_data.drop(columns=['diabetes'], axis=1)
B = diabetes_data['diabetes']

print(A)
print(B)

       gender   age  hypertension  heart_disease smoking_history    bmi  \
0      Female  80.0             0              1           never  25.19   
1      Female  54.0             0              0         No Info  27.32   
2        Male  28.0             0              0           never  27.32   
3      Female  36.0             0              0         current  23.45   
4        Male  76.0             1              1         current  20.14   
...       ...   ...           ...            ...             ...    ...   
99995  Female  80.0             0              0         No Info  27.32   
99996  Female   2.0             0              0         No Info  17.37   
99997    Male  66.0             0              0          former  27.83   
99998  Female  24.0             0              0           never  35.42   
99999  Female  57.0             0              0         current  22.43   

       HbA1c_level  blood_glucose_level  
0              6.6                  140  
1              

In [130]:
#encoding non numeric values
#Gender
A['gender'] = A['gender'].replace({'Female': 0, 'Male': 1, 'Other': 2})

#Smoking History
A['smoking_history'] = A['smoking_history'].replace({'No Info': 0, 'never': 1, 'current': 2, 'former': 3, 'ever': 4, 'not current': 5})

print(A)


       gender   age  hypertension  heart_disease  smoking_history    bmi  \
0           0  80.0             0              1                1  25.19   
1           0  54.0             0              0                0  27.32   
2           1  28.0             0              0                1  27.32   
3           0  36.0             0              0                2  23.45   
4           1  76.0             1              1                2  20.14   
...       ...   ...           ...            ...              ...    ...   
99995       0  80.0             0              0                0  27.32   
99996       0   2.0             0              0                0  17.37   
99997       1  66.0             0              0                3  27.83   
99998       0  24.0             0              0                1  35.42   
99999       0  57.0             0              0                2  22.43   

       HbA1c_level  blood_glucose_level  
0              6.6                  140  
1  

<ipython-input-130-84c4d85b3948>:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  A['gender'] = A['gender'].replace({'Female': 0, 'Male': 1, 'Other': 2})
<ipython-input-130-84c4d85b3948>:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  A['smoking_history'] = A['smoking_history'].replace({'No Info': 0, 'never': 1, 'current': 2, 'former': 3, 'ever': 4, 'not current': 5})


Data encoding for string values

Gender

Female: 0

Male: 1

Other: 2


Smoking History


No Info: 0


Never smoked: 1

Current smoker: 2

Former smoker: 3

Has ever smoked before: 4

Not currently smoking: 5

Data Preprocessing

In [131]:
#Data standardization
scaler = StandardScaler()

#Fit calculates the mean and standard deviation and transform uses that data to
# help keep values between 0 and 1
A = scaler.fit_transform(A)

print(A)

[[-8.41046744e-01  1.69270354e+00 -2.84439447e-01 ... -3.21055792e-01
   1.00170572e+00  4.77042159e-02]
 [-8.41046744e-01  5.38006427e-01 -2.84439447e-01 ... -1.15583678e-04
   1.00170572e+00 -1.42620999e+00]
 [ 1.18723364e+00 -6.16690686e-01 -2.84439447e-01 ... -1.15583678e-04
   1.61108022e-01  4.89878478e-01]
 ...
 [ 1.18723364e+00  1.07094356e+00 -2.84439447e-01 ...  7.67292549e-02
   1.61108022e-01  4.16182767e-01]
 [-8.41046744e-01 -7.94336396e-01 -2.84439447e-01 ...  1.22036126e+00
  -1.42668764e+00 -9.34905254e-01]
 [-8.41046744e-01  6.71240710e-01 -2.84439447e-01 ... -7.36921977e-01
   1.00170572e+00 -1.18055762e+00]]


Train Test Splitting

In [132]:
a_train, a_test, b_train, b_test = train_test_split(A, B, test_size=0.2, stratify=B, random_state=2)
print(A.shape, a_train.shape, a_test.shape)

(100000, 8) (80000, 8) (20000, 8)


Train the model

In [133]:
#Instantate the classifier
model = svm.SVC(kernel='linear')

#Train the model
model.fit(a_train, b_train)

SVC(kernel='linear')

Model Evaluation: Testing model accuracy

In [134]:
#Check accuracy of training data
a_train_prediction = model.predict(a_train)
training_data_accuracy = accuracy_score(a_train_prediction, b_train)

print('Accuracy score of the training data: ', training_data_accuracy)

Accuracy score of the training data:  0.960425


In [135]:
#Check accuracy score of the testing data
a_test_prediction = model.predict(a_test)
testing_data_accuracy = accuracy_score(a_test_prediction, b_test)

print('Accuracy score of the testing data: ', testing_data_accuracy)

Accuracy score of the testing data:  0.9617


Predictive system using trained model

In [136]:
#This patient is diabetic. This cell checks the model's accuracy
input = (1,69.0,0,1,0,29.34,8.2,155)

#As numpy array
input_np = np.asarray(input)

#Reshape array to get prediction for just one instance
input_np_reshaped = input_np.reshape(1, -1)

#Standardize input data
std_data = scaler.transform(input_np_reshaped)
print(std_data)

prediction = model.predict(std_data)
print(prediction)

if (prediction[0] == 0):
  print('The person is not diabetic')
else:
  print('The person is diabetic')

[[ 1.18723364  1.20417784 -0.28443945  4.93637859 -0.89116652  0.30425025
   2.49610164  0.41618277]]
[1]
The person is diabetic


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Packaging the model with pickle

In [138]:
import pickle

with open('/content/diabetes_prediction_model.pkl', 'wb') as f:
  pickle.dump(model, f)